1. Есть некоторое количество точек на плоскости, для которых известны взаимные расстояния, равные $\sqrt{E_i*E_j}r_{ij}$, $r_{ij}$ - евклидовое расстояние.  
2. Точки сортируются по убыванию энергии.  
2. Берём самую высокоэнергичную частицу и присоединяем к ней ближайшую при условии, что расстояние между ними < $\chi_{ij}$.  
3. Точки объединяются в одну, для них пересчитывается расстояние (энергетически взвешенно  $X_{new} = \frac{E_iX_i + E_jX_j}{E_i+E_j}$) и энергия.  
4. Переходим к следующей точке, пока расстояния между всеми точками меньше $\chi_{ij}$ или точки не закончатся.  

In [661]:
import pandas as pd
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import math

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import AgglomerativeClustering
import datetime
le = LabelEncoder()

from platform import python_version

print(python_version())

3.7.9


In [583]:
# Функция подсчета значений коэффициентов

def CoefPFI_analysis(df, num, cluster_column, hlabel_column):
    P = 0
    E = 0
    DomH = [] #доминирующие высоты в кластерах
    N_c = len(df)
    
    labels = df[cluster_column]
    
    n_clusters_ = len(set(labels))
    
    for i in range(n_clusters_):
        
        value = df[df[cluster_column]==i][hlabel_column].value_counts()

        n_d = value[:1].values
        n_c = len(df[df[cluster_column]==i][hlabel_column])
        
        
        P_i = n_d[0]/n_c
        P = P + P_i
        
        x = value[:1].index[0]
        n_t = len(df[df[hlabel_column]==x])
        
        E_i = n_d[0]/n_t
        E = E + E_i
        DomH.append(x)
    
    purity = P/n_clusters_ #1
    
    splitting = len(set(DomH))/n_clusters_ #2
    
    integrity = len(set(DomH))/len(set(df[hlabel_column])) #3
 
    efficiency =  E/n_clusters_ #4
    
    print(f'P = {round(purity,3)}, S = {round(splitting,3)}, I = {round(integrity, 3)}, E = {round(efficiency, 3)}')
   
    
    result = [ num, round(purity, 3), round(splitting,3), round(integrity, 3), round(efficiency,3)] 
    mean = np.mean(result[1:])
    var = np.var(result[1:])
    compose = (purity+efficiency)*(integrity+splitting)/4
    
    result = result + [mean]
    result = result +[var]
    
    
    print(f'Mean: {round(mean, 3)}, compose: {round( compose , 3)}')
    
    return 

In [594]:
def Matrix_of_Distance(X, Y, E, label):
    
    # out = попарное евклидовое расстояние
    Z = np.array([[complex(X[i], Y[i]) for i in range(len(X))]])
    out = abs(Z.T-Z)
    
    if label==1:
        # евклидовое расстояние
        oute = 1
    elif label==2:
        # sqrt(e_i*e_j)
        oute = np.sqrt(np.outer(E, E))
    elif label==3:
        # e_i*e_j/(e_i+e_j)
        outmult = np.outer(E, E)
        outadd = np.add.outer(list(E), list(E))
        oute = outmult/outadd
        
    # Для треугольной формы    
    # oute = np.triu(oute)
    return pd.DataFrame(oute*out)

In [144]:
# Считываем данные модели
AllMc0 = pd.read_csv('datachanged/AllMc0C').drop(['Unnamed: 0'], axis = 1)

In [675]:
OneFamily = pd.DataFrame(AllMc0.loc[lambda AllMc0: AllMc0[' num_of_family'] == 5, :]).copy()
OneFamily['Hlabel'] = le.fit_transform(OneFamily['H(J)'])

In [676]:
OneFamily

,num_of_family,j,X(J),Y(J),E(J),H(J),A0,R,cnt,ER,sum_energy,lg_r,lg_e_r,Hlabel
101,5,1,-2.362921,-0.449419,11.454860,1098.068,1,2.405280,11,27.552149,299.830038,0.381166,1.440155,0
102,5,2,-1.582896,-0.379217,11.953560,1098.068,1,1.627687,11,19.456656,299.830038,0.211571,1.289068,0
103,5,3,-1.686270,-1.102141,5.687628,1098.068,1,2.014503,11,11.457742,299.830038,0.304168,1.059099,0
104,5,4,-1.247690,0.033633,4.312032,1098.068,1,1.248143,11,5.382033,299.830038,0.096264,0.730946,0
105,5,5,-1.604474,-0.220549,4.245429,1098.068,1,1.619561,11,6.875733,299.830038,0.209397,0.837319,0
106,5,6,-1.731634,-1.100501,4.644425,1098.068,1,2.051745,11,9.529177,299.830038,0.312123,0.979055,0
107,5,7,-0.358915,-0.165094,6.386136,1098.068,1,0.395065,11,2.522939,299.830038,-0.403331,0.401907,0
108,5,8,0.313507,-1.810078,6.507324,1098.068,1,1.837027,11,11.954131,299.830038,0.264116,1.077518,0
109,5,9,0.039927,-0.263272,6.106994,1098.068,1,0.266282,11,1.626184,299.830038,-0.574658,0.211170,0
110,5,10,-5.231096,-0.644797,5.315850,45614.860,1,5.270686,11,28.018175,299.830038,0.721867,1.447440,1


In [744]:
def PamirAlgorithmZ(OneFamily, 
                   chiri = 48,  
                   zcri = 3.6,
                   label = 2,
                   num_name = ' num_of_family',
                   j_name = ' j',
                   x_name = 'X(J)',
                   y_name = 'Y(J)',
                   e_name = 'E(J)'):
    
    """
    Алгоритм предыдущей кластеризации.
    Две метрика для label = 2 и label = 3.
    Возвращает номера кластеров для точек семейства.
    """

    if label==2:
        eps = chiri
    elif label==3:
        epc = zcri
    else:
        print('label error')
        return

    # создадим словарь кластеров, так как нумерация частиц может быть некорректной
    points = OneFamily[j_name].unique()
    clusters = {}
    for k in points:
        clusters[k]=-1

    # Уберём названия колонок
    # Необходимый порядок:
    # 0 - номер семейства 1 - индекс частицы 2 - координата x 3 - координата y 4 - энергия 
    dfv = OneFamily[[num_name, j_name, x_name, y_name, e_name]].values
    dftmp = pd.DataFrame(dfv)

    # отсортируем значения по энергии
    dftmp = dftmp.sort_values(by = 4, ascending = False, ignore_index = True)
    # добавим значение расстояния от самой энергетичной частицы до остальных
    dftmp[5] = [-1]*len(dftmp)
    # Добавим переменную для остановки, когда все частицы кластеризованы
    check = len(dftmp)
    #print('check =', check)
    k=0

    #идём по циклу, пока каждая частица не окажется в кластере
    while check>0:    
        dfv = dftmp.values

        # координаты и энергии высокоэн частицы
        i = dfv[0:1][0][1]
        xi = dfv[0:1][0][2]
        yi = dfv[0:1][0][3]
        ei = dfv[0:1][0][4]

        # рассматриваем новую частицу - новый кластер
        if i!=-1:
            #print("i = ", i, 'k = ', k)
            clusters[i] = k

        # присваиваем частице индекс -1
        dfv[0:1][0][1] = -1
        dftmp.iloc[0, 1] = -1
        
        # идём по номерам частиц
        for j in range(1,len(dftmp)+1):
            # считаем для всех элементов массива расстояния до самой энергичной частицы
            xj = dfv[j-1:j][0][2]
            yj = dfv[j-1:j][0][3]
            ej = dfv[j-1:j][0][4]

            if label==2:
                ri = float(np.sqrt(ei*ej*((xi-xj)**2+(yi-yj)**2)))
            if label==3:
                ri = float((ei*ej/(ei+ej))*np.sqrt((xi-xj)**2+(yi-yj)**2))

            # находим индекс ближайшего к самой энергичной частице
            # minri - значение минимального расстояния
            # j_min - индекс минимального расстояния во временной матрице
            # i_min - номер частицы в семействе (максимальный номер может не совпадать с количеством частиц из-за того что что-то убиралось)
            if j<=2:
                minri = ri
                j_min = j
                i_min = int(dfv[j-1:j][0][1])
            elif (j>2) and (ri<minri):
                minri = ri
                j_min = j
                i_min = int(dfv[j-1:j][0][1])        
            # обновляем значение столбца матрицы взаимных расстояний
            dfv[j-1:j][0][5] = ri 
        # конец цикла, нашли minri
        # print('i_min =', i_min)
        # print('minri = ', minri)

        #проверяем, что значение расстояния меньше критерия остановки
        if (minri<=eps):
            # Считаем координаты и энергию "перевзвешенной" самой энергичной частицы
            ximin = dfv[j_min-1:j_min][0][2]
            yimin = dfv[j_min-1:j_min][0][3]
            eimin = dfv[j_min-1:j_min][0][4]
            dfv[0:1][0][2] = (ei*xi+eimin*ximin)/(ei+eimin)
            dfv[0:1][0][3] = (ei*yi+eimin*yimin)/(ei+eimin)
            dfv[0:1][0][4] = (ei+eimin)

            # clusters[i_min] = k

            # Переопределяем матрицу расстояний
            dftmp = pd.DataFrame(dfv)
            # print('drop ', i_min)
            dftmp = dftmp[dftmp[1]!=i_min].copy().reset_index(drop=True)
            check = len(dftmp)

            # для последней точки ближайшей может быть она сама
            if (i_min==-1) and (i!=-1):
                #print("!!!!!!!!!!!", i)
                clusters[i] = k
            elif (i_min!=-1):
                #print("i_min = ", i_min, 'k = ', k)
                clusters[i_min] = k
            #print('check =', check)
        else:
            # если расстояние превышено, кластер выбывает
            # уже отсортировано по энергии, самая энергияная частица всегда первая
            # print('new cluster drop ', -1)
            dftmp = dftmp[dftmp[1]!=-1].copy().reset_index(drop=True)
            check = len(dftmp)

            # print('check =', check)
            k+=1  

    # labels - спискок из кластеров идёт по порядку словаря, а словарб по порядку j 
    labels = clusters.values()
    return labels
    #return clusters

In [746]:
OneFamily['clust_pam']= PamirAlgorithmZ(OneFamily)
CoefPFI_analysis(OneFamily, ' num_of_fam', 'Hlabel', 'clust_pam')

P = 0.583, S = 1.0, I = 1.0, E = 0.554
Mean: 0.784, compose: 0.568


In [747]:
chiri = np.percentile(OneFamily['ER'],50)
OneFamily['clust_pam']= PamirAlgorithmZ(OneFamily, chiri)
CoefPFI_analysis(OneFamily, ' num_of_fam', 'Hlabel', 'clust_pam')

P = 0.528, S = 1.0, I = 0.4, E = 1.0
Mean: 0.732, compose: 0.535


In [748]:
chiri

11.457742200424962

In [653]:
X = Matrix_of_Distance(OneFamily['X(J)'].values, OneFamily['Y(J)'].values, OneFamily['E(J)'].values, 2)

In [654]:
#linkage: average, complete, single.
thr = np.percentile(OneFamily, 50)
clustering = AgglomerativeClustering(n_clusters=None, affinity = 'precomputed', distance_threshold = thr, linkage = 'average').fit(X)
clustering.labels_

array([ 7,  8,  6,  4,  3,  1,  5,  4,  4,  3,  5,  1,  1,  5,  5,  9,  0,
       11,  0,  2,  2,  2,  2,  2, 10, 10], dtype=int64)

In [655]:
OneFamily['clust_agl']= clustering.labels_

In [656]:
CoefPFI_analysis(OneFamily, ' num_of_fam', 'Hlabel', 'clust_agl')

P = 0.69, S = 1.0, I = 0.583, E = 0.869
Mean: 0.785, compose: 0.617


In [754]:
def clustering(data, name, exp = False):
    
    nums =list(set(data[name].values))
    clustered_df = []
    
    print('start: ', datetime.datetime.now())
    
    for i in nums:
        if i%100 == 0:
            print(i, datetime.datetime.now())
        
        df = data[data[name]==i].copy()
        
        # eps = 48
        df['cluster_pam'] = PamirAlgorithm(df)
        
        clustered_df.append(df)
    
    clustered_df = pd.concat(clustered_df)

    return clustered_df

In [755]:
# Старый модельный банк.
olddata = pd.read_csv('datachanged/AllMc0COld', sep = '\t')

In [756]:
# Экспериментальные данные.
AllExp = pd.read_csv('datachanged/AllExpC')
AllExp = AllExp.drop(['Unnamed: 0'], axis = 1)

In [757]:
clustered_df_old = clustering(olddata, ' num_of_family')

start:  2022-03-07 00:51:48.703287
100 2022-03-07 00:51:52.088278
200 2022-03-07 00:51:56.700944
300 2022-03-07 00:51:59.675991
400 2022-03-07 00:52:02.595608
500 2022-03-07 00:52:05.507822
600 2022-03-07 00:52:08.511792
700 2022-03-07 00:52:12.119147
800 2022-03-07 00:52:17.792979
900 2022-03-07 00:52:22.715820
1000 2022-03-07 00:52:26.077831
1100 2022-03-07 00:52:29.942497
1200 2022-03-07 00:52:33.877976


In [758]:
clustered_df_old

,num_of_family,j,X(J),Y(J),E(J),H(J),E0,A0,log_E0,R,ER,sum_energy,cluster_pam
0,1,1,-3.013189,-7.227429,4.611031,1255.2500,3366.712,1,3.527206,7.830392,36.106180,370.199507,4
1,1,2,-1.310529,-2.563248,12.047700,1545.0540,3366.712,1,3.527206,2.878841,34.683415,370.199507,1
2,1,3,-2.178048,-2.144561,5.401844,3044.5200,3366.712,1,3.527206,3.056638,16.511481,370.199507,1
3,1,4,-0.965465,-0.082880,5.325506,4715.5230,3366.712,1,3.527206,0.969016,5.160501,370.199507,0
4,1,5,-1.621645,0.079540,15.935060,3044.5200,3366.712,1,3.527206,1.623595,25.872076,370.199507,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27153,1298,6,-1.070832,-0.511355,5.577535,678.2601,25863.250,9,4.412683,1.186661,6.618646,226.805048,0
27154,1298,7,-0.093955,-0.075777,64.149700,678.2601,25863.250,9,4.412683,0.120705,7.743186,226.805048,0
27155,1298,8,-0.564117,-0.356699,27.180820,678.2601,25863.250,9,4.412683,0.667429,18.141268,226.805048,0
27156,1298,9,2.566935,2.136236,8.525503,45614.5600,25863.250,9,4.412683,3.339560,28.471427,226.805048,3
